In [1]:
import requests
import pandas as pd
import json

# Get your API key at https://api.census.gov/data/key_signup.html
census_api_key = "ENTER API KEY"
# Get your API key at https://data.bls.gov/registrationEngine/
bls_api_key = "ENTER API KEY"


def get_acs_b_or_c_table(year, table, for_string, census_api_key, acs_1_3_or_5 = 5, debug = False):
    """Pull a B or C ACS table from the API
    Args:
        year (int): The ACS year you want to pull
        table (str): The table name
        for_string (str): The geography component of the API call
        acs_1_3_or_5 (int): Do you want a 1, 3 or 5 year estimate (optional default 5)
        debug (bool): Enable debug mode
    Returns:
        df (DataFrame): Pandas Data Frame with the requested data (false on failure)
    """
    api_base = "https://api.census.gov/data/" + str(year) + "/acs/acs" + str(acs_1_3_or_5) + "?get="
    api_vars = "group(" + table + ")"
    if year == 2016:
        api_vars = api_vars + ",NAME,GEO_ID"
    api_key = "&key=" + census_api_key
    api_url = api_base + api_vars + for_string + api_key
    if debug:
        print(api_url)
    response = requests.get(api_url)
    try:
        df = pd.DataFrame.from_dict(response.json())
        # Pull the column names from the first record and drop it (it's not data)
        df = df.rename(columns = df.iloc[0])
        df = df.iloc[1:].reset_index(drop=True)
        # Drop the MA and EA fields
        df = df.loc[:,~df.columns.str.contains("MA")]
        df = df.loc[:,~df.columns.str.contains("EA")]
        return df
    except:
        print("ERROR: ACS Pull Failed")
        print(json.loads(response.text))
        return False

    
def get_cpi(year, bls_api_key, debug = False):
    """Pulls the annual CPI-U for a given year.
    Args:
        year (int): The year you want the CPI for
        bls_api_key (str): The API key
        debug (bool): Use debug mode (optional default False)
    Returns:
        cpi (float): The CPI-U (false on failure)
    """
    headers = {"Content-type": "application/json"}
    data = json.dumps({"seriesid":['CUUR0000SA0'], 
                       "startyear":year, 
                       "endyear":year,
                       "annualaverage":True,
                       "registrationkey": bls_api_key})
    response = requests.post("https://api.bls.gov/publicAPI/v2/timeseries/data/", data=data, headers=headers)
    json_data = json.loads(response.text)
    if debug:
        print(json_data)
    try:
        df = pd.DataFrame(json_data["Results"]["series"][0]["data"])
        cpi = float(df[df.period == "M13"]['value'])
        return cpi
    except:
        print("Something bad happened")
        print(json_data)
        return False
    
    
def inflation_adjust(dollar_value, dollar_cpi, target_cpi):
    """Takes a dollar value and adjusts it for inflation.
    Args:
        dollar_value (int or float): The dollar value you want to inflation adjust
        dollar_cpi (float): The CPI that corresponds to the year of the dollar value
        target_cpi (float): The CPI for the year you want the dollars to be in
    Returns:
        inflation_adjusted_dollar_value (int): The dollar value that has been inflation adjusted
    """
    inflation_adjustment_factor = target_cpi / dollar_cpi
    inflation_adjusted_dollar_value = int(round(dollar_value * inflation_adjustment_factor, 0))
    return inflation_adjusted_dollar_value


mhi_tables = {"Total": "B19013", "White": "B19013A", "Black or African-American": "B19013B", "Asian": "B19013D", "Hispanic":"B19013I", "Male": "B19326_002E", "Female": "B19326_005E"}
for_strings = ["&for=place:63000&in=state:36",
               "&for=county:055&in=state:36",
               "&for=state:36",
               "&for=us:1"]

years = range(2013, 2019)

# We want the dollar values to be inflation adjusted so they represent the same purchasing power.
latest_year = max(years)
# Pull the CPI for the year we want to inflate the dollars to match
cpis = {"Target": get_cpi(latest_year, bls_api_key)}

# This will hold the results
mhi_data = []

# Loop through the parameters to pull all the data
for year in years:
    print("Pulling data for " + str(year))
    for group, mhi_table in mhi_tables.items():
        for for_string in for_strings:
            if group == "Male" or group == "Female":
                table = mhi_table.split("_")
                acs_data = get_acs_b_or_c_table(year, table[0], for_string, census_api_key, acs_1_3_or_5 = 1)
                mhi = int(acs_data[mhi_table])
            else:
                acs_data = get_acs_b_or_c_table(year, mhi_table, for_string, census_api_key, acs_1_3_or_5 = 1)
                mhi = int(acs_data[mhi_table+"_001E"])
            name = acs_data["NAME"][0]
            geo_id = acs_data["GEO_ID"][0]
            if year == latest_year:
                inflation_adjusted_mhi = mhi
            else:
                # Check to see if we have the CPI
                if not year in cpis.keys():
                    cpis[year] = get_cpi(year, bls_api_key)
                inflation_adjusted_mhi = inflation_adjust(mhi, cpis[year], cpis["Target"])
            row = {"ID": geo_id, "Name": name, "MHI": mhi, "Inflation Adjusted MHI": inflation_adjusted_mhi, "Group": group, "Year": year}
            mhi_data.append(row)
            
mhi_data = pd.DataFrame(mhi_data)

Pulling data for 2013
Pulling data for 2014
Pulling data for 2015
Pulling data for 2016
Pulling data for 2017
Pulling data for 2018


In [2]:
mhi_data

,ID,Name,MHI,Inflation Adjusted MHI,Group,Year
0,1600000US3663000,"Rochester city, New York",30158,32508,Total,2013
1,0500000US36055,"Monroe County, New York",51665,55690,Total,2013
2,0400000US36,New York,57369,61839,Total,2013
3,0100000US,United States,52250,56321,Total,2013
4,1600000US3663000,"Rochester city, New York",38166,41140,White,2013
...,...,...,...,...,...,...
163,0100000US,United States,37669,37669,Male,2018
164,1600000US3663000,"Rochester city, New York",20629,20629,Female,2018
165,0500000US36055,"Monroe County, New York",25877,25877,Female,2018
166,0400000US36,New York,28180,28180,Female,2018


In [3]:
mhi_data.to_excel("MHI.xlsx", index = False)